In [1]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 11.0 MB/s eta 0:00:0000:010:01


In [19]:
import pandas as pd
import random
import json
from faker import Faker

def extract_and_generate_csv():
    fake = Faker()
    
    # Load the original CSV files
    movies_path = "tmdb_5000_movies.csv"
    theaters_path = "theaters.csv"
    
    movies_df = pd.read_csv(movies_path)
    theaters_df = pd.read_csv(theaters_path)
    
    # Generate new unique IDs for movies and theatres
    movies_df['movie_id'] = range(1000, 1000 + len(movies_df))
    theaters_df['theatre_id'] = range(5000, 5000 + len(theaters_df))
    
    # Extract relevant columns from Movies dataset
    def extract_production_house(production_companies):
        try:
            companies = json.loads(production_companies.replace("'", "\""))
            return companies[0]['name'] if companies else fake.company()
        except:
            return fake.company()
    
    movies_selected = movies_df[['movie_id', 'title', 'vote_average', 'release_date', 'runtime', 'production_companies']].copy()
    movies_selected = movies_selected.drop(columns=["production_companies"])
    movies_selected.rename(columns={
        'title': 'name',
        'vote_average': 'rating',
        'release_date': 'release_date',
        'runtime': 'run_time'
    }, inplace=True)
    
    movies_selected['production_house'] = movies_df['production_companies'].apply(extract_production_house)
    
    # Fill missing values with fake data
    movies_selected.fillna({
        'rating': round(random.uniform(5.0, 9.0), 1),
        'release_date': fake.date(),
        'run_time': random.randint(80, 180)
    }, inplace=True)
    
    # Extract relevant columns from Theatres dataset
    theaters_selected = theaters_df[['theatre_id', 'name', 'address', 'city', 'state', 'zip', 'lat', 'lon']].copy()
    theaters_selected.rename(columns={
        'name': 'theatre_name',
        'lat': 'latitude',
        'lon': 'longitude'
    }, inplace=True)
    
    # Generate random screens data (max 20 per theatre)
    screens_data = []
    for _, row in theaters_selected.iterrows():
        num_screens = random.randint(5, 20)
        for screen_no in range(1, num_screens + 1):
            screens_data.append({
                'screen_id': f"{row['theatre_id']}_{screen_no}",
                'theatre_id': row['theatre_id'],
                'screen_no': screen_no,
                'screen_type': random.choice(['IMAX', '3D', 'Standard', '4DX', 'Dolby Cinema']),
                'capacity': random.randint(50, 300)
            })
    
    screens_df = pd.DataFrame(screens_data)
    
    # Create Screening Schedule
    schedule_data = []
    for _, screen_row in screens_df.iterrows():
        num_movies = random.randint(1, 3)  # Each screen can play up to 3 different movies a day
        movies_sample = movies_selected.sample(num_movies)  # Randomly select movies for this screen
        for _, movie_row in movies_sample.iterrows():
            show_times = ['10:00 AM', '01:00 PM', '04:00 PM', '07:00 PM', '10:00 PM']
            for show_time in random.sample(show_times, random.randint(2, 4)):  # Assign 2-4 random showtimes
                schedule_data.append({
                    'schedule_id': f"{screen_row['screen_id']}_{movie_row['movie_id']}_{show_time.replace(':', '')}",
                    'screen_id': screen_row['screen_id'],
                    'movie_id': movie_row['movie_id'],
                    'show_date': fake.date_this_year().strftime('%Y-%m-%d'),  # Random show date
                    'show_time': show_time,
                    'available_seats': screen_row['capacity']
                })
    
    schedule_df = pd.DataFrame(schedule_data)
    
    # Save extracted data to CSV files
    movies_selected.to_csv("Movies.csv", index=False)
    theaters_selected.to_csv("Theatres.csv", index=False)
    screens_df.to_csv("Screens.csv", index=False)
    schedule_df.to_csv("Screening_Schedule.csv", index=False)
    
    print("CSV files have been successfully created and are ready for direct database insertion!")

# Run the function
extract_and_generate_csv()


CSV files have been successfully created and are ready for direct database insertion!


In [21]:
import pandas as pd
import random
from faker import Faker

def generate_customer_transactions():
    fake = Faker()
    
    # Load the generated CSV files
    movies_df = pd.read_csv("Movies.csv")
    theaters_df = pd.read_csv("Theatres.csv")
    screens_df = pd.read_csv("Screens.csv")
    schedule_df = pd.read_csv("Screening_Schedule.csv")
    
    # Generate customer transactions
    transactions_data = []
    num_transactions = 100000 
    
    for _ in range(num_transactions):
        schedule_sample = schedule_df.sample(1).iloc[0]  # Select a random screening
        seats_booked = random.randint(1, 5)  # Random number of seats booked
        ticket_price = random.choice([10, 12, 15, 20])  # Random ticket price
        food_amount = random.choice([0, 5, 10, 15, 20])  # Food purchase amount
        total_amount = (seats_booked * ticket_price) + food_amount
        
        transactions_data.append({
            'transaction_id': fake.uuid4(),
            'customer_name': fake.name(),
            'email_id': fake.email(),
            'promo_code_used': random.choice([None, 'DISCOUNT10', 'SUMMER20', 'MOVIE50']),
            'schedule_id': schedule_sample['schedule_id'],
            'seats_booked': seats_booked,
            'ticket_price': ticket_price,
            'food_bought': random.choice([None, 'Popcorn', 'Soda', 'Nachos', 'Hotdog']),
            'food_amount': food_amount,
            'total_amount': total_amount,
            'booking_date': fake.date_this_year().strftime('%Y-%m-%d')
        })
    
    transactions_df = pd.DataFrame(transactions_data)
    
    # Save to CSV
    transactions_df.to_csv("Customer_Transactions.csv", index=False)
    
    print("Customer Transactions CSV file has been successfully created!")

# Run the function
generate_customer_transactions()


Customer Transactions CSV file has been successfully created!


In [23]:
import pandas as pd
import random
from faker import Faker

def generate_theatre_staff_and_shifts():
    fake = Faker()
    
    # Load the generated Theatres CSV file
    theaters_df = pd.read_csv("Theatres.csv")
    
    # Generate Theatre Staff
    staff_data = []
    roles = ['Manager', 'Cashier', 'Usher', 'Cleaner', 'Security', 'Projectionist']
    
    staff_id_counter = 1
    for _, theatre in theaters_df.iterrows():
        num_staff = random.randint(5, 15)  # Random number of staff per theatre
        for _ in range(num_staff):
            staff_data.append({
                'staff_id': staff_id_counter,
                'theatre_id': theatre['theatre_id'],
                'name': fake.name(),
                'role': random.choice(roles),
                'salary': round(random.uniform(25000, 70000), 2),
                'contact_no': fake.phone_number(),
                'email': fake.email()
            })
            staff_id_counter += 1
    
    staff_df = pd.DataFrame(staff_data)
    
    # Generate Daily Shifts
    shifts_data = []
    shift_id_counter = 1
    
    for _, staff in staff_df.iterrows():
        num_shifts = random.randint(20, 30)  # Each staff works around 20-30 shifts per month
        for _ in range(num_shifts):
            shifts_data.append({
                'shift_id': shift_id_counter,
                'staff_id': staff['staff_id'],
                'shift_date': fake.date_this_year().strftime('%Y-%m-%d'),
                'shift_start_time': fake.time(pattern="%H:%M:%S"),
                'shift_end_time': fake.time(pattern="%H:%M:%S")
            })
            shift_id_counter += 1
    
    shifts_df = pd.DataFrame(shifts_data)
    
    # Save to CSV
    staff_df.to_csv("Theatre_Staff.csv", index=False)
    shifts_df.to_csv("Daily_Shifts.csv", index=False)
    
    print("Theatre Staff and Daily Shifts CSV files have been successfully created!")

# Run the function
generate_theatre_staff_and_shifts()


Theatre Staff and Daily Shifts CSV files have been successfully created!


In [25]:
import pandas as pd
import random
from faker import Faker

def normalize_and_insert():
    fake = Faker()
    
    # Load existing datasets
    movies_df = pd.read_csv("Movies.csv")
    theaters_df = pd.read_csv("Theatres.csv")
    screens_df = pd.read_csv("Screens.csv")
    schedule_df = pd.read_csv("Screening_Schedule.csv")
    transactions_df = pd.read_csv("Customer_Transactions.csv")
    staff_df = pd.read_csv("Theatre_Staff.csv")
    shifts_df = pd.read_csv("Daily_Shifts.csv")
    
    # 1. Customer Table (Separate customer details)
    customer_data = {}
    for _, transaction in transactions_df.iterrows():
        if transaction['email_id'] not in customer_data:
            customer_data[transaction['email_id']] = {
                'customer_id': len(customer_data) + 1,
                'customer_name': transaction['customer_name'],
                'email_id': transaction['email_id'],
                'contact_no': fake.phone_number()
            }
    customer_df = pd.DataFrame(customer_data.values())
    
    # 2. Transaction Table (Link customers to transactions separately)
    transaction_data = []
    for _, transaction in transactions_df.iterrows():
        customer_id = customer_df.loc[customer_df['email_id'] == transaction['email_id'], 'customer_id'].values[0]
        transaction_data.append({
            'transaction_id': transaction['transaction_id'],
            'customer_id': customer_id,
            'schedule_id': transaction['schedule_id'],
            'seats_booked': transaction['seats_booked'],
            'ticket_price': transaction['ticket_price'],
            'food_amount': transaction['food_amount'],
            'total_amount': transaction['total_amount'],
            'booking_date': transaction['booking_date']
        })
    transactions_normalized_df = pd.DataFrame(transaction_data)
    
    # 3. Food Details (Break down food purchases per transaction)
    food_items = ['Popcorn', 'Soda', 'Nachos', 'Hotdog', 'Burger', 'Candy']
    food_details_data = []
    for _, transaction in transactions_df.iterrows():
        if pd.notna(transaction['food_bought']):
            for food in transaction['food_bought'].split(', '):
                food_details_data.append({
                    'transaction_id': transaction['transaction_id'],
                    'food_name': food,
                    'price': round(random.uniform(3.0, 10.0), 2)
                })
    food_details_df = pd.DataFrame(food_details_data)
    
    # 4. Promotions Table (Store promo codes and discounts separately)
    promo_codes = ['DISCOUNT10', 'SUMMER20', 'MOVIE50', None]
    promotions_data = []
    for _, transaction in transactions_df.iterrows():
        if transaction['promo_code_used'] in promo_codes[:-1]:  # Exclude None values
            promotions_data.append({
                'transaction_id': transaction['transaction_id'],
                'promo_code': transaction['promo_code_used'],
                'discount_applied': round(random.uniform(5.0, 20.0), 2)
            })
    promotions_df = pd.DataFrame(promotions_data)
    
    # Save new tables
    customer_df.to_csv("Customers.csv", index=False)
    transactions_normalized_df.to_csv("Transactions.csv", index=False)
    food_details_df.to_csv("Food_Details.csv", index=False)
    promotions_df.to_csv("Promotions.csv", index=False)
    
    print("New tables Customers, Transactions, Food_Details, and Promotions have been created and data inserted!")

# Run the function
normalize_and_insert()


New tables Customers, Transactions, Food_Details, and Promotions have been created and data inserted!


In [43]:
!pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 6.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.10-cp312-cp312-macosx_11_0_arm64.whl size=133409 sha256=8903b8f3db086689d66b4c013aedd139f85fab8907c5e7a683f5ee45d630d355
  Stored in directory: /Users/vamsisaigarapati/Library/Caches/pip/wheels/ac/bb/ce/afa589c50b6004d3a06fc691e71bd09c9bd5f01e5921e5329b
Successfully built psycopg2


In [51]:
import mysql.connector
from mysql.connector import errorcode
import psycopg2
def create_database_and_tables():
    try:
        conn = psycopg2.connect( # Database name
    user="postgres",  # Your username
    password="NbKhArDy369#",  # Your password
    host="localhost",  # Or the host of your database
    port="5432"  # Or the port if it's different
)
        # Connect to MySQL Server
        connection = mysql.connector.connect(
            host='localhost',
            user='postgres',
            password='NbKhArDy369@'
        )
        cursor = connection.cursor()
        
        # Create Database
        cursor.execute("CREATE DATABASE IF NOT EXISTS MovieChainDB")
        cursor.execute("USE MovieChainDB")
        
        # Define Table Creation Queries
        TABLES = {}
        TABLES['Movies'] = (
            "CREATE TABLE IF NOT EXISTS Movies ("
            "  movie_id INT AUTO_INCREMENT PRIMARY KEY,"
            "  name VARCHAR(255) NOT NULL,"
            "  rating DECIMAL(3,1) NOT NULL,"
            "  production_house VARCHAR(255) NOT NULL,"
            "  release_date DATE NOT NULL,"
            "  run_time INT NOT NULL CHECK (run_time > 0)"
            ")"
        )
        
        TABLES['Theatres'] = (
            "CREATE TABLE IF NOT EXISTS Theatres ("
            "  theatre_id INT AUTO_INCREMENT PRIMARY KEY,"
            "  name VARCHAR(255) NOT NULL,"
            "  address VARCHAR(255) NOT NULL,"
            "  city VARCHAR(100) NOT NULL,"
            "  state VARCHAR(50) NOT NULL,"
            "  zip VARCHAR(20) NOT NULL,"
            "  language VARCHAR(50) NOT NULL,"
            "  latitude DECIMAL(10,6) NOT NULL,"
            "  longitude DECIMAL(10,6) NOT NULL"
            ")"
        )
        
        TABLES['Screens'] = (
            "CREATE TABLE IF NOT EXISTS Screens ("
            "  screen_id INT AUTO_INCREMENT PRIMARY KEY,"
            "  theatre_id INT NOT NULL,"
            "  screen_no INT NOT NULL,"
            "  screen_type VARCHAR(50) NOT NULL,"
            "  capacity INT NOT NULL CHECK (capacity > 0),"
            "  FOREIGN KEY (theatre_id) REFERENCES Theatres(theatre_id) ON DELETE CASCADE"
            ")"
        )
        
        TABLES['Screening_Schedule'] = (
            "CREATE TABLE IF NOT EXISTS Screening_Schedule ("
            "  schedule_id INT AUTO_INCREMENT PRIMARY KEY,"
            "  screen_id INT NOT NULL,"
            "  movie_id INT NOT NULL,"
            "  show_date DATE NOT NULL,"
            "  show_time TIME NOT NULL,"
            "  available_seats INT NOT NULL CHECK (available_seats >= 0),"
            "  FOREIGN KEY (screen_id) REFERENCES Screens(screen_id) ON DELETE CASCADE,"
            "  FOREIGN KEY (movie_id) REFERENCES Movies(movie_id) ON DELETE CASCADE"
            ")"
        )
        
        TABLES['Customers'] = (
            "CREATE TABLE IF NOT EXISTS Customers ("
            "  customer_id INT AUTO_INCREMENT PRIMARY KEY,"
            "  customer_name VARCHAR(255) NOT NULL,"
            "  email_id VARCHAR(255) UNIQUE NOT NULL,"
            "  contact_no VARCHAR(20) NOT NULL"
            ")"
        )
        
        TABLES['Transactions'] = (
            "CREATE TABLE IF NOT EXISTS Transactions ("
            "  transaction_id INT AUTO_INCREMENT PRIMARY KEY,"
            "  customer_id INT NOT NULL,"
            "  schedule_id INT NOT NULL,"
            "  seats_booked INT NOT NULL CHECK (seats_booked > 0),"
            "  ticket_price DECIMAL(5,2) NOT NULL,"
            "  food_amount DECIMAL(5,2) NOT NULL DEFAULT 0,"
            "  total_amount DECIMAL(6,2) NOT NULL,"
            "  booking_date DATETIME NOT NULL,"
            "  FOREIGN KEY (customer_id) REFERENCES Customers(customer_id) ON DELETE CASCADE,"
            "  FOREIGN KEY (schedule_id) REFERENCES Screening_Schedule(schedule_id) ON DELETE CASCADE"
            ")"
        )
        
        TABLES['Food_Details'] = (
            "CREATE TABLE IF NOT EXISTS Food_Details ("
            "  food_id INT AUTO_INCREMENT PRIMARY KEY,"
            "  transaction_id INT NOT NULL,"
            "  food_name VARCHAR(100) NOT NULL,"
            "  price DECIMAL(5,2) NOT NULL CHECK (price > 0),"
            "  FOREIGN KEY (transaction_id) REFERENCES Transactions(transaction_id) ON DELETE CASCADE"
            ")"
        )
        
        TABLES['Promotions'] = (
            "CREATE TABLE IF NOT EXISTS Promotions ("
            "  promo_id INT AUTO_INCREMENT PRIMARY KEY,"
            "  transaction_id INT NOT NULL,"
            "  promo_code VARCHAR(50) NOT NULL,"
            "  discount_applied DECIMAL(5,2) NOT NULL CHECK (discount_applied > 0),"
            "  FOREIGN KEY (transaction_id) REFERENCES Transactions(transaction_id) ON DELETE CASCADE"
            ")"
        )
        
        TABLES['Theatre_Staff'] = (
            "CREATE TABLE IF NOT EXISTS Theatre_Staff ("
            "  staff_id INT AUTO_INCREMENT PRIMARY KEY,"
            "  theatre_id INT NOT NULL,"
            "  name VARCHAR(255) NOT NULL,"
            "  role VARCHAR(50) NOT NULL,"
            "  salary DECIMAL(8,2) NOT NULL CHECK (salary > 0),"
            "  contact_no VARCHAR(20) NOT NULL,"
            "  email VARCHAR(255) UNIQUE NOT NULL,"
            "  FOREIGN KEY (theatre_id) REFERENCES Theatres(theatre_id) ON DELETE CASCADE"
            ")"
        )
        
        TABLES['Daily_Shifts'] = (
            "CREATE TABLE IF NOT EXISTS Daily_Shifts ("
            "  shift_id INT AUTO_INCREMENT PRIMARY KEY,"
            "  staff_id INT NOT NULL,"
            "  shift_date DATE NOT NULL,"
            "  shift_start_time TIME NOT NULL,"
            "  shift_end_time TIME NOT NULL,"
            "  FOREIGN KEY (staff_id) REFERENCES Theatre_Staff(staff_id) ON DELETE CASCADE"
            ")"
        )
        
        # Execute table creation queries
        for table_name, table_query in TABLES.items():
            cursor.execute(table_query)
        
        print("Database and tables created successfully!")
        
        # Close connection
        cursor.close()
        connection.close()
        
    except mysql.connector.Error as err:
        print(f"Error: {err}")

# Run the function
create_database_and_tables()


Error: 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (61)


In [55]:
import psycopg2

# Connection details for Server 1
DB_NAME = "postgres"
DB_USER = "postgres"
DB_PASSWORD = "NbKhArDy369@"
DB_HOST = "localhost"  # Change to the correct host IP
DB_PORT = "5432"  # Default PostgreSQL port

try:
    # Connect to PostgreSQL
    connection = psycopg2.connect(
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        host=DB_HOST,
        port=DB_PORT
    )
    print("Connected to PostgreSQL Server 1 successfully!")

    # Close connection
    connection.close()

except psycopg2.Error as e:
    print(f"Error connecting to PostgreSQL: {e}")

Connected to PostgreSQL Server 1 successfully!


In [55]:
import psycopg2
from psycopg2 import sql

def create_database():
    try:
        # Connect to PostgreSQL server (default 'postgres' database)
        connection = psycopg2.connect(
            dbname='postgres',  # Connect to the default database first
            user='postgres',
            password='NbKhArDy369@',
            host='localhost',  # Change if connecting to a remote server
            port='5432'
        )
        connection.autocommit = True  # Required for creating databases
        cursor = connection.cursor()
        
        # Create the database if it does not exist
        cursor.execute("CREATE DATABASE movie_chain_db3;")
        print("Database 'movie_chain_db3' created successfully!")
        
        cursor.close()
        connection.close()
    except psycopg2.Error as e:
        print(f"Error creating database: {e}")

def create_tables():
    try:
        # Connect to the new database
        connection = psycopg2.connect(
            dbname='movie_chain_db3',
            user='postgres',
            password='NbKhArDy369@',
            host='localhost',
            port='5432'
        )
        cursor = connection.cursor()
        
        # List of SQL create table statements
        TABLES = [
            """
            CREATE TABLE IF NOT EXISTS Movies (
                movie_id SERIAL PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                rating DECIMAL(3,1) NOT NULL,
                production_house VARCHAR(255) NOT NULL,
                release_date DATE NOT NULL,
                run_time INT NOT NULL CHECK (run_time > 0)
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS Theatres (
                theatre_id SERIAL PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                address VARCHAR(255) NOT NULL,
                city VARCHAR(100) NOT NULL,
                state VARCHAR(50) NOT NULL,
                zip VARCHAR(20) NOT NULL,
                latitude DECIMAL(10,6) NOT NULL,
                longitude DECIMAL(10,6) NOT NULL
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS Screens (
                screen_id SERIAL PRIMARY KEY,
                theatre_id INT NOT NULL,
                screen_no INT NOT NULL,
                screen_type VARCHAR(50) NOT NULL,
                capacity INT NOT NULL CHECK (capacity > 0),
                FOREIGN KEY (theatre_id) REFERENCES Theatres(theatre_id) ON DELETE CASCADE
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS Screening_Schedule (
                schedule_id SERIAL PRIMARY KEY,
                screen_id INT NOT NULL,
                movie_id INT NOT NULL,
                show_date DATE NOT NULL,
                show_time TIME NOT NULL,
                available_seats INT NOT NULL CHECK (available_seats >= 0),
                FOREIGN KEY (screen_id) REFERENCES Screens(screen_id) ON DELETE CASCADE,
                FOREIGN KEY (movie_id) REFERENCES Movies(movie_id) ON DELETE CASCADE
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS Customers (
                customer_id SERIAL PRIMARY KEY,
                customer_name VARCHAR(255) NOT NULL,
                email_id VARCHAR(255) UNIQUE NOT NULL,
                contact_no VARCHAR(100) NOT NULL
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS Transactions (
                transaction_id SERIAL PRIMARY KEY,
                customer_id INT NOT NULL,
                schedule_id INT NOT NULL,
                seats_booked INT NOT NULL CHECK (seats_booked > 0),
                ticket_price DECIMAL(5,2) NOT NULL,
                food_amount DECIMAL(5,2) NOT NULL DEFAULT 0,
                total_amount DECIMAL(6,2) NOT NULL,
                booking_date TIMESTAMP NOT NULL,
                FOREIGN KEY (customer_id) REFERENCES Customers(customer_id) ON DELETE CASCADE,
                FOREIGN KEY (schedule_id) REFERENCES Screening_Schedule(schedule_id) ON DELETE CASCADE
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS Food_Details (
                food_id SERIAL PRIMARY KEY,
                transaction_id INT NOT NULL,
                food_name VARCHAR(100) NOT NULL,
                price DECIMAL(5,2) NOT NULL CHECK (price > 0),
                FOREIGN KEY (transaction_id) REFERENCES Transactions(transaction_id) ON DELETE CASCADE
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS Promotions (
                promo_id SERIAL PRIMARY KEY,
                transaction_id INT NOT NULL,
                promo_code VARCHAR(50) NOT NULL,
                discount_applied DECIMAL(5,2) NOT NULL CHECK (discount_applied > 0),
                FOREIGN KEY (transaction_id) REFERENCES Transactions(transaction_id) ON DELETE CASCADE
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS Theatre_Staff (
                staff_id SERIAL PRIMARY KEY,
                theatre_id INT NOT NULL,
                name VARCHAR(255) NOT NULL,
                role VARCHAR(50) NOT NULL,
                salary DECIMAL(8,2) NOT NULL CHECK (salary > 0),
                contact_no VARCHAR(100) NOT NULL,
                email VARCHAR(255) UNIQUE NOT NULL,
                FOREIGN KEY (theatre_id) REFERENCES Theatres(theatre_id) ON DELETE CASCADE
            )
            """,
            """
            CREATE TABLE IF NOT EXISTS Daily_Shifts (
                shift_id SERIAL PRIMARY KEY,
                staff_id INT NOT NULL,
                shift_date DATE NOT NULL,
                shift_start_time TIME NOT NULL,
                shift_end_time TIME NOT NULL,
                FOREIGN KEY (staff_id) REFERENCES Theatre_Staff(staff_id) ON DELETE CASCADE
            )
            """
        ]
        
        # Execute table creation queries
        for table_query in TABLES:
            cursor.execute(table_query)
        
        print("All tables created successfully!")
        
        # Commit changes and close connection
        connection.commit()
        cursor.close()
        connection.close()
    except psycopg2.Error as e:
        print(f"Error creating tables: {e}")

# Run the functions
create_database()
create_tables()

Database 'movie_chain_db3' created successfully!
All tables created successfully!


In [65]:
import pandas as pd

# Load original CSV files
movies_df = pd.read_csv("Movies.csv")
theaters_df = pd.read_csv("Theatres.csv")
screens_df = pd.read_csv("Screens.csv")
schedule_df = pd.read_csv("Screening_Schedule.csv")
transactions_df = pd.read_csv("Transactions.csv")
staff_df = pd.read_csv("Theatre_Staff.csv")
shifts_df = pd.read_csv("Daily_Shifts.csv")
food_df = pd.read_csv("Food_Details.csv")
promotions_df = pd.read_csv("Promotions.csv")
customers_df = pd.read_csv("Customers.csv")

# Assign new auto-incremented IDs while maintaining relationships
movies_df["new_movie_id"] = range(1, len(movies_df) + 1)
theaters_df["new_theatre_id"] = range(1, len(theaters_df) + 1)
screens_df["new_screen_id"] = range(1, len(screens_df) + 1)
schedule_df["new_schedule_id"] = range(1, len(schedule_df) + 1)
transactions_df["new_transaction_id"] = range(1, len(transactions_df) + 1)
staff_df["new_staff_id"] = range(1, len(staff_df) + 1)
shifts_df["new_shift_id"] = range(1, len(shifts_df) + 1)
customers_df["new_customer_id"] = range(1, len(customers_df) + 1)
food_df["new_food_id"] = range(1, len(food_df) + 1)
promotions_df["new_promo_id"] = range(1, len(promotions_df) + 1)

# Create mapping dictionaries to update foreign keys
movie_id_map = movies_df.set_index("movie_id")["new_movie_id"].to_dict()
theatre_id_map = theaters_df.set_index("theatre_id")["new_theatre_id"].to_dict()
screen_id_map = screens_df.set_index("screen_id")["new_screen_id"].to_dict()
schedule_id_map = schedule_df.set_index("schedule_id")["new_schedule_id"].to_dict()
transaction_id_map = transactions_df.set_index("transaction_id")["new_transaction_id"].to_dict()
staff_id_map = staff_df.set_index("staff_id")["new_staff_id"].to_dict()
customer_id_map = customers_df.set_index("customer_id")["new_customer_id"].to_dict()

# Update foreign keys in related tables
screens_df["theatre_id"] = screens_df["theatre_id"].map(theatre_id_map)
schedule_df["screen_id"] = schedule_df["screen_id"].map(screen_id_map)
schedule_df["movie_id"] = schedule_df["movie_id"].map(movie_id_map)
transactions_df["schedule_id"] = transactions_df["schedule_id"].map(schedule_id_map)
transactions_df["customer_id"] = transactions_df["customer_id"].map(customer_id_map)
staff_df["theatre_id"] = staff_df["theatre_id"].map(theatre_id_map)
shifts_df["staff_id"] = shifts_df["staff_id"].map(staff_id_map)
food_df["transaction_id"] = food_df["transaction_id"].map(transaction_id_map)
promotions_df["transaction_id"] = promotions_df["transaction_id"].map(transaction_id_map)

# Drop old ID columns and rename new IDs
movies_df.drop(columns=["movie_id"], inplace=True)
movies_df.rename(columns={"new_movie_id": "movie_id"}, inplace=True)

theaters_df.drop(columns=["theatre_id"], inplace=True)
theaters_df.rename(columns={"new_theatre_id": "theatre_id"}, inplace=True)

screens_df.drop(columns=["screen_id"], inplace=True)
screens_df.rename(columns={"new_screen_id": "screen_id"}, inplace=True)

schedule_df.drop(columns=["schedule_id"], inplace=True)
schedule_df.rename(columns={"new_schedule_id": "schedule_id"}, inplace=True)

transactions_df.drop(columns=["transaction_id"], inplace=True)
transactions_df.rename(columns={"new_transaction_id": "transaction_id"}, inplace=True)

staff_df.drop(columns=["staff_id"], inplace=True)
staff_df.rename(columns={"new_staff_id": "staff_id"}, inplace=True)

shifts_df.drop(columns=["shift_id"], inplace=True)
shifts_df.rename(columns={"new_shift_id": "shift_id"}, inplace=True)

customers_df.drop(columns=["customer_id"], inplace=True)
customers_df.rename(columns={"new_customer_id": "customer_id"}, inplace=True)

# food_df.drop(columns=["food_id"], inplace=True)
# food_df.rename(columns={"new_food_id": "food_id"}, inplace=True)

# promotions_df.drop(columns=["promo_id"], inplace=True)
# promotions_df.rename(columns={"new_promo_id": "promo_id"}, inplace=True)

# Save the updated CSV files
movies_df.to_csv("Movies_Updated.csv", index=False)
theaters_df.to_csv("Theatres_Updated.csv", index=False)
screens_df.to_csv("Screens_Updated.csv", index=False)
schedule_df.to_csv("Screening_Schedule_Updated.csv", index=False)
transactions_df.to_csv("Transactions_Updated.csv", index=False)
staff_df.to_csv("Theatre_Staff_Updated.csv", index=False)
shifts_df.to_csv("Daily_Shifts_Updated.csv", index=False)
customers_df.to_csv("Customers_Updated.csv", index=False)
food_df.to_csv("Food_Details_Updated.csv", index=False)
promotions_df.to_csv("Promotions_Updated.csv", index=False)

print("Updated CSV files with correct ID mappings and relationships!")

Updated CSV files with correct ID mappings and relationships!


In [ ]:
import psycopg2
import pandas as pd
import datetime

def validate_and_insert_data():
    try:
        # Connect to the PostgreSQL database
        connection = psycopg2.connect(
            dbname='movie_chain_db',
            user='postgres',
            password='NbKhArDy369@',
            host='localhost',
            port='5432'
        )
        cursor = connection.cursor()
        
        # Load data from updated CSV files
        movies_df = pd.read_csv("Movies_Updated.csv")
        theaters_df = pd.read_csv("Theatres_Updated.csv")
        screens_df = pd.read_csv("Screens_Updated.csv")
        schedule_df = pd.read_csv("Screening_Schedule_Updated.csv")
        transactions_df = pd.read_csv("Transactions_Updated.csv")
        staff_df = pd.read_csv("Theatre_Staff_Updated.csv")
        shifts_df = pd.read_csv("Daily_Shifts_Updated.csv")
        customers_df = pd.read_csv("Customers_Updated.csv")
        food_df = pd.read_csv("Food_Details_Updated.csv")
        promotions_df = pd.read_csv("Promotions_Updated.csv")
        
        # Data validation and constraint enforcement
        movies_df["run_time"] = movies_df["run_time"].apply(lambda x: x if x > 0 else 90)
        screens_df["capacity"] = screens_df["capacity"].apply(lambda x: x if x > 0 else 50)
        schedule_df["available_seats"] = schedule_df["available_seats"].apply(lambda x: x if x >= 0 else 0)
        transactions_df["seats_booked"] = transactions_df["seats_booked"].apply(lambda x: x if x > 0 else 1)
        food_df["price"] = food_df["price"].apply(lambda x: x if x > 0 else 5.0)
        promotions_df["discount_applied"] = promotions_df["discount_applied"].apply(lambda x: x if x > 0 else 0.0)
        staff_df["salary"] = staff_df["salary"].apply(lambda x: x if x > 0 else 30000)
        
        # Insert data into Movies
        for _, row in movies_df.iterrows():
            cursor.execute("""
                INSERT INTO Movies (movie_id, name, rating, production_house, release_date, run_time)
                VALUES (%s, %s, %s, %s, %s, %s)""",
                (row['movie_id'], row['name'], row['rating'], row['production_house'], row['release_date'], row['run_time'])
            )
        
        # Insert data into Theatres
        
        
        # Close connection
        cursor.close()
        connection.close()
    except psycopg2.Error as e:
        print(f"Error inserting data into PostgreSQL: {e}")

# Run the function
validate_and_insert_data()


In [29]:
import pandas as pd

def adjust_csv_columns():
    # Define the correct column order as per SQL tables
    column_orders = {
        # "Movies_Updated.csv": ["movie_id", "name", "rating", "production_house", "release_date", "run_time"],
        # "Theatres_Updated.csv": ["theatre_id", "name", "address", "city", "state", "zip", "latitude", "longitude"],
        # "Screens_Updated.csv": ["screen_id", "theatre_id", "screen_no", "screen_type", "capacity"],
        # "Screening_Schedule_Updated.csv": ["schedule_id", "screen_id", "movie_id", "show_date", "show_time", "available_seats"],
        # "Customers_Updated.csv": ["customer_id", "customer_name", "email_id", "contact_no"],
        # "Transactions_Updated.csv": ["transaction_id", "customer_id", "schedule_id", "seats_booked", "ticket_price", "food_amount", "total_amount", "booking_date"],
        
        # "Food_Details_Updated.csv": ["food_id", "transaction_id", "food_name", "price"],
        # "Promotions_Updated.csv": ["promo_id", "transaction_id", "promo_code", "discount_applied"],
        # transaction_id	promo_code	discount_applied	new_promo_id
        # "Theatre_Staff_Updated.csv": ["staff_id", "theatre_id", "name", "role", "salary", "contact_no", "email"],
        # "Daily_Shifts_Updated.csv": ["shift_id", "staff_id", "shift_date", "shift_start_time", "shift_end_time"]
    }
    
    for file_name, correct_order in column_orders.items():
        try:
            # Read CSV file
            df = pd.read_csv(file_name)
            # df = df.rename(columns={"new_promo_id": "promo_id"})
            
            # Adjust column names if different from the expected ones
            df.columns = [col.lower().replace(" ", "_") for col in df.columns]  # Normalize column names
            correct_order_lower = [col.lower() for col in correct_order]
            
            # Reorder columns according to the table
            df = df[correct_order]
            
            # Save updated CSV
            df.to_csv(file_name, index=False)
            print(f"Updated {file_name} with correct column order and names.")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

# Run the function
adjust_csv_columns()


Updated Daily_Shifts_Updated.csv with correct column order and names.


In [31]:
import pandas as pd

def convert_csv_to_pgadmin_txt(csv_file_path, output_txt_path):
    try:
        # Load the CSV file
        df = pd.read_csv(csv_file_path)
        
        # Ensure proper data formatting
        if "movie_id" in df.columns:
            df["movie_id"] = pd.to_numeric(df["movie_id"], errors="coerce").fillna(0).astype(int)
        
        if "rating" in df.columns:
            df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
        
        if "run_time" in df.columns:
            df["run_time"] = pd.to_numeric(df["run_time"], errors="coerce").fillna(0).astype(int)
        
        if "release_date" in df.columns:
            df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce").dt.strftime('%Y-%m-%d')
        
        # Save to a text file with tab delimiters (better for pgAdmin import)
        df.to_csv(output_txt_path, sep='\t', index=False, header=True, encoding="utf-8")
        
        print(f"File successfully converted: {output_txt_path}")
    except Exception as e:
        print(f"Error processing file: {e}")

# Example usage
csv_file_path = "Movies_Updated.csv"  # Change this to your actual file path
output_txt_path = "Movies_Formatted.txt"
convert_csv_to_pgadmin_txt(csv_file_path, output_txt_path)

File successfully converted: Movies_Formatted.txt


/var/folders/x5/7fmrypvd2mj6rc45kq6kv2lh0000gn/T/ipykernel_92651/4011215736.py:19: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["release_date"] = pd.to_datetime(df["release_date"], errors="coerce").dt.strftime('%Y-%m-%d')


In [37]:
import psycopg2
import pandas as pd

def insert_csv_to_pgsql(csv_file_path, table_name):
    try:
        # Connect to PostgreSQL database
        connection = psycopg2.connect(
            dbname='movie_chain_db',
            user='postgres',
            password='NbKhArDy369@',
            host='localhost',
            port='5432'
        )
        cursor = connection.cursor()
        
        # Load CSV file
        df = pd.read_csv(csv_file_path)
        df["run_time"] = df["run_time"].apply(lambda x: x if x > 0 else 90)
        # screens_df["capacity"] = screens_df["capacity"].apply(lambda x: x if x > 0 else 50)
        # schedule_df["available_seats"] = schedule_df["available_seats"].apply(lambda x: x if x >= 0 else 0)
        # transactions_df["seats_booked"] = transactions_df["seats_booked"].apply(lambda x: x if x > 0 else 1)
        # food_df["price"] = food_df["price"].apply(lambda x: x if x > 0 else 5.0)
        # promotions_df["discount_applied"] = promotions_df["discount_applied"].apply(lambda x: x if x > 0 else 0.0)
        # staff_df["salary"] = staff_df["salary"].apply(lambda x: x if x > 0 else 30000)

        # Ensure column names match database schema
        df.columns = [col.lower().replace(" ", "_") for col in df.columns]

        # Convert all NumPy data types to native Python types
        df = df.astype(object)

        # Convert dataframe to list of tuples for batch insertion
        records_list = df.to_records(index=False).tolist()

        # Create SQL query dynamically
        columns = ", ".join(df.columns)
        placeholders = ", ".join(["%s"] * len(df.columns))
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

        # Use executemany for batch insert
        cursor.executemany(query, records_list)

        # Commit transaction
        connection.commit()
        print(f"Data from {csv_file_path} inserted into {table_name} successfully!")

        # Close connection
        cursor.close()
        connection.close()
    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

# Example usage
insert_csv_to_pgsql("Movies_Updated.csv", "movies")

Data from Movies_Updated.csv inserted into movies successfully!


In [57]:
import psycopg2
import pandas as pd

def insert_csv_to_pgsql(csv_file_path, table_name, constraints=None):
    try:
        # Connect to PostgreSQL database
        connection = psycopg2.connect(
            dbname='movie_chain_db3',
            user='postgres',
            password='NbKhArDy369@',
            host='localhost',
            port='5432'
        )
        cursor = connection.cursor()
        
        # Load CSV file
        df = pd.read_csv(csv_file_path)

        # Ensure column names match database schema
        df.columns = [col.lower().replace(" ", "_") for col in df.columns]

        # Convert all NumPy data types to native Python types
        df = df.astype(object)

        # Apply constraints if provided
        if constraints:
            for column, condition in constraints.items():
                df[column] = df[column].apply(condition)
        
        # Convert dataframe to list of tuples for batch insertion
        records_list = df.to_records(index=False).tolist()

        # Create SQL query dynamically
        columns = ", ".join(df.columns)
        placeholders = ", ".join(["%s"] * len(df.columns))
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

        # Use executemany for batch insert
        cursor.executemany(query, records_list)

        # Commit transaction
        connection.commit()
        print(f"Data from {csv_file_path} inserted into {table_name} successfully!")

        # Close connection
        cursor.close()
        connection.close()
    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

# Define constraints for each table
constraints_dict = {
    "movies": {
        "run_time": lambda x: x if x > 0 else 90
    },
    "screens": {
        "capacity": lambda x: x if x > 0 else 50
    },
    "screening_schedule": {
        "available_seats": lambda x: x if x >= 0 else 0
    },
    "transactions": {
        "seats_booked": lambda x: x if x > 0 else 1
    },
    "food_details": {
        "price": lambda x: x if x > 0 else 5.0
    },
    "promotions": {
        "discount_applied": lambda x: x if x > 0 else 0.0
    },
    "theatre_staff": {
        "salary": lambda x: x if x > 0 else 30000
    }
}

# Insert data for all tables
insert_csv_to_pgsql("Movies_Updated.csv", "movies", constraints_dict.get("movies"))
insert_csv_to_pgsql("Theatres_Updated.csv", "theatres")
insert_csv_to_pgsql("Screens_Updated.csv", "screens", constraints_dict.get("screens"))
insert_csv_to_pgsql("Screening_Schedule_Updated.csv", "screening_schedule", constraints_dict.get("screening_schedule"))
insert_csv_to_pgsql("Customers_Updated.csv", "customers")
insert_csv_to_pgsql("Transactions_Updated.csv", "transactions", constraints_dict.get("transactions"))
insert_csv_to_pgsql("Food_Details_Updated.csv", "food_details", constraints_dict.get("food_details"))
insert_csv_to_pgsql("Promotions_Updated.csv", "promotions", constraints_dict.get("promotions"))
insert_csv_to_pgsql("Theatre_Staff_Updated.csv", "theatre_staff", constraints_dict.get("theatre_staff"))
insert_csv_to_pgsql("Daily_Shifts_Updated.csv", "daily_shifts")

Data from Movies_Updated.csv inserted into movies successfully!
Data from Theatres_Updated.csv inserted into theatres successfully!
Data from Screens_Updated.csv inserted into screens successfully!
Data from Screening_Schedule_Updated.csv inserted into screening_schedule successfully!
Data from Customers_Updated.csv inserted into customers successfully!
Data from Transactions_Updated.csv inserted into transactions successfully!
Data from Food_Details_Updated.csv inserted into food_details successfully!
Data from Promotions_Updated.csv inserted into promotions successfully!
Error inserting data into theatre_staff: duplicate key value violates unique constraint "theatre_staff_email_key"
DETAIL:  Key (email)=(greenjennifer@example.net) already exists.

Error inserting data into daily_shifts: insert or update on table "daily_shifts" violates foreign key constraint "daily_shifts_staff_id_fkey"
DETAIL:  Key (staff_id)=(1) is not present in table "theatre_staff".



In [59]:
import psycopg2
import pandas as pd

def remove_duplicates(df, unique_columns):
    """Remove duplicate rows based on unique constraints"""
    return df.drop_duplicates(subset=unique_columns, keep='first')

def insert_csv_to_pgsql(csv_file_path, table_name, unique_columns=None, constraints=None):
    try:
        # Connect to PostgreSQL database
        connection = psycopg2.connect(
            dbname='movie_chain_db3',
            user='postgres',
            password='NbKhArDy369@',
            host='localhost',
            port='5432'
        )
        cursor = connection.cursor()
        
        # Load CSV file
        df = pd.read_csv(csv_file_path)

        # Ensure column names match database schema
        df.columns = [col.lower().replace(" ", "_") for col in df.columns]

        # Convert all NumPy data types to native Python types
        df = df.astype(object)

        # Apply constraints if provided
        if constraints:
            for column, condition in constraints.items():
                df[column] = df[column].apply(condition)
        
        # Remove duplicates based on unique constraints
        if unique_columns:
            df = remove_duplicates(df, unique_columns)
        
        # Convert dataframe to list of tuples for batch insertion
        records_list = df.to_records(index=False).tolist()

        # Create SQL query dynamically
        columns = ", ".join(df.columns)
        placeholders = ", ".join(["%s"] * len(df.columns))
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders}) ON CONFLICT DO NOTHING"

        # Use executemany for batch insert
        cursor.executemany(query, records_list)

        # Commit transaction
        connection.commit()
        print(f"Data from {csv_file_path} inserted into {table_name} successfully, duplicates removed!")

        # Close connection
        cursor.close()
        connection.close()
    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

# Define constraints and unique columns for each table
constraints_dict = {
    "movies": {
        "run_time": lambda x: x if x > 0 else 90
    },
    "screens": {
        "capacity": lambda x: x if x > 0 else 50
    },
    "screening_schedule": {
        "available_seats": lambda x: x if x >= 0 else 0
    },
    "transactions": {
        "seats_booked": lambda x: x if x > 0 else 1
    },
    "food_details": {
        "price": lambda x: x if x > 0 else 5.0
    },
    "promotions": {
        "discount_applied": lambda x: x if x > 0 else 0.0
    },
    "theatre_staff": {
        "salary": lambda x: x if x > 0 else 30000
    }
}

unique_columns_dict = {
    "theatre_staff": ["email"],
    "daily_shifts": ["staff_id", "shift_date"]
}

# Insert data for all tables with duplicate handling
# insert_csv_to_pgsql("Movies_Updated.csv", "movies", constraints_dict.get("movies"))
# insert_csv_to_pgsql("Theatres_Updated.csv", "theatres")
# insert_csv_to_pgsql("Screens_Updated.csv", "screens", constraints_dict.get("screens"))
# insert_csv_to_pgsql("Screening_Schedule_Updated.csv", "screening_schedule", constraints_dict.get("screening_schedule"))
# insert_csv_to_pgsql("Customers_Updated.csv", "customers")
# insert_csv_to_pgsql("Transactions_Updated.csv", "transactions", constraints_dict.get("transactions"))
# insert_csv_to_pgsql("Food_Details_Updated.csv", "food_details", constraints_dict.get("food_details"))
# insert_csv_to_pgsql("Promotions_Updated.csv", "promotions", constraints_dict.get("promotions"))
insert_csv_to_pgsql("Theatre_Staff_Updated.csv", "theatre_staff", unique_columns_dict.get("theatre_staff"), constraints_dict.get("theatre_staff"))
insert_csv_to_pgsql("Daily_Shifts_Updated.csv", "daily_shifts", unique_columns_dict.get("daily_shifts"))


Data from Theatre_Staff_Updated.csv inserted into theatre_staff successfully, duplicates removed!
Error inserting data into daily_shifts: insert or update on table "daily_shifts" violates foreign key constraint "daily_shifts_staff_id_fkey"
DETAIL:  Key (staff_id)=(181) is not present in table "theatre_staff".



In [61]:
import psycopg2
import pandas as pd
import random
import string

def generate_unique_email(existing_emails, base_email):
    """Generate a new unique email by appending a random string."""
    while base_email in existing_emails:
        random_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=4))
        base_email = base_email.split('@')[0] + random_suffix + "@example.com"
    existing_emails.add(base_email)
    return base_email

def clean_and_update_csv(csv_file_path, unique_columns, save_path):
    """Modify CSV file to remove duplicates and ensure unique constraints."""
    df = pd.read_csv(csv_file_path)
    
    # Ensure column names match schema
    df.columns = [col.lower().replace(" ", "_") for col in df.columns]
    
    # Handle duplicate emails for theatre_staff
    if "email" in df.columns:
        existing_emails = set()
        df["email"] = df["email"].apply(lambda email: generate_unique_email(existing_emails, email))
    
    # Remove duplicate rows based on unique constraints
    df = df.drop_duplicates(subset=unique_columns, keep='first')
    
    # Save the cleaned CSV file
    df.to_csv(save_path, index=False)
    print(f"Updated CSV saved: {save_path}")

def insert_csv_to_pgsql(csv_file_path, table_name, unique_columns=None, constraints=None):
    try:
        # Connect to PostgreSQL database
        connection = psycopg2.connect(
            dbname='movie_chain_db3',
            user='postgres',
            password='NbKhArDy369@',
            host='localhost',
            port='5432'
        )
        cursor = connection.cursor()
        
        # Load CSV file
        df = pd.read_csv(csv_file_path)
        
        # Ensure column names match database schema
        df.columns = [col.lower().replace(" ", "_") for col in df.columns]
        
        # Convert all NumPy data types to native Python types
        df = df.astype(object)
        
        # Apply constraints if provided
        if constraints:
            for column, condition in constraints.items():
                df[column] = df[column].apply(condition)
        
        # Convert dataframe to list of tuples for batch insertion
        records_list = df.to_records(index=False).tolist()
        
        # Create SQL query dynamically
        columns = ", ".join(df.columns)
        placeholders = ", ".join(["%s"] * len(df.columns))
        query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders}) ON CONFLICT DO NOTHING"
        
        # Use executemany for batch insert
        cursor.executemany(query, records_list)
        
        # Commit transaction
        connection.commit()
        print(f"Data from {csv_file_path} inserted into {table_name} successfully, duplicates handled!")
        
        # Close connection
        cursor.close()
        connection.close()
    except Exception as e:
        print(f"Error inserting data into {table_name}: {e}")

# Define constraints and unique columns for each table
constraints_dict = {
    "movies": {
        "run_time": lambda x: x if x > 0 else 90
    },
    "screens": {
        "capacity": lambda x: x if x > 0 else 50
    },
    "screening_schedule": {
        "available_seats": lambda x: x if x >= 0 else 0
    },
    "transactions": {
        "seats_booked": lambda x: x if x > 0 else 1
    },
    "food_details": {
        "price": lambda x: x if x > 0 else 5.0
    },
    "promotions": {
        "discount_applied": lambda x: x if x > 0 else 0.0
    },
    "theatre_staff": {
        "salary": lambda x: x if x > 0 else 30000
    }
}

unique_columns_dict = {
    "theatre_staff": ["email"],
    "daily_shifts": ["staff_id", "shift_date"]
}

# Clean and update theatre_staff CSV before inserting
clean_and_update_csv("Theatre_Staff_Updated.csv", unique_columns_dict.get("theatre_staff"), "Theatre_Staff_Cleaned.csv")

# Insert data for all tables with duplicate handling
# insert_csv_to_pgsql("Movies_Updated.csv", "movies", constraints_dict.get("movies"))
# insert_csv_to_pgsql("Theatres_Updated.csv", "theatres")
# insert_csv_to_pgsql("Screens_Updated.csv", "screens", constraints_dict.get("screens"))
# insert_csv_to_pgsql("Screening_Schedule_Updated.csv", "screening_schedule", constraints_dict.get("screening_schedule"))
# insert_csv_to_pgsql("Customers_Updated.csv", "customers")
# insert_csv_to_pgsql("Transactions_Updated.csv", "transactions", constraints_dict.get("transactions"))
# insert_csv_to_pgsql("Food_Details_Updated.csv", "food_details", constraints_dict.get("food_details"))
# insert_csv_to_pgsql("Promotions_Updated.csv", "promotions", constraints_dict.get("promotions"))
insert_csv_to_pgsql("Theatre_Staff_Cleaned.csv", "theatre_staff", unique_columns_dict.get("theatre_staff"), constraints_dict.get("theatre_staff"))
insert_csv_to_pgsql("Daily_Shifts_Updated.csv", "daily_shifts", unique_columns_dict.get("daily_shifts"))


Updated CSV saved: Theatre_Staff_Cleaned.csv
Data from Theatre_Staff_Cleaned.csv inserted into theatre_staff successfully, duplicates handled!
Data from Daily_Shifts_Updated.csv inserted into daily_shifts successfully, duplicates handled!
